In [1]:
import sys, pathlib
sys.path.insert(0, str(pathlib.Path.cwd().parent.parent))  # add repo root
from tsum import tsum
import torch
import json

from ndtools import fun_binary_graph as fbg # ndtools available at github.com/jieunbyun/network-datasets
from ndtools.graphs import build_graph
from pathlib import Path
import networkx as nx   

# Load data

In [2]:
DATASET = Path("data") 

nodes = json.loads((DATASET / "nodes.json").read_text(encoding="utf-8"))
edges = json.loads((DATASET / "edges.json").read_text(encoding="utf-8"))
probs_dict = json.loads((DATASET / "probs_eq.json").read_text(encoding="utf-8"))

# build base graph
G_base: nx.Graph = build_graph(nodes, edges, probs_dict)


In [3]:
row_names = list(edges.keys())
n_state = 2 # binary states

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
probs = [[probs_dict[n]['0']['p'], probs_dict[n]['1']['p']] for n in row_names]
probs = torch.tensor(probs, dtype=torch.float32, device=device)

In [4]:
dests = ['n22', 'n66']
sys_surv_st = 2

def s_fun(comps_st):
    conn_pop_ratio, sys_st, info = fbg.eval_population_accessibility(comps_st, G_base, dests,
                                                         avg_speed=60.0, # km/h
                                                         target_time_max = 0.25, # hours: it shouldn't take longer than this to reach any destination
                                                         target_pop_max = [0.80, 0.95], # fraction of population that should be reachable at each destination
                                                         length_attr = 'length_km',
                                                         population_attr = 'population',)

    min_comps_st = None
    return conn_pop_ratio, sys_st, min_comps_st

row_names = list(edges.keys()) 
n_state = 2 # binary states of components

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
probs = [[probs_dict[n]['0']['p'], probs_dict[n]['1']['p']] for n in row_names]
probs = torch.tensor(probs, dtype=torch.float32, device=device)


In [5]:
TSUMPATH = Path("tsum_res") 

rules_mat_surv = torch.load(TSUMPATH / f"rules_geq_{sys_surv_st}.pt", map_location="cpu")
rules_mat_surv = rules_mat_surv.to(device)
rules_mat_fail = torch.load(TSUMPATH / f"rules_leq_{sys_surv_st-1}.pt", map_location="cpu")
rules_mat_fail = rules_mat_fail.to(device)

# Calculate system probabilities

## Marginal probability

In [6]:
pr_cond = tsum.get_comp_cond_sys_prob(
    rules_mat_surv,
    rules_mat_fail,
    probs,
    comps_st_cond = {},
    row_names = row_names,
    s_fun = s_fun,
    sys_surv_st = sys_surv_st
)
print(f"P(sys >= {sys_surv_st}) = {pr_cond['survival']:.3e}")
print(f"P(sys <= {sys_surv_st-1} ) = {pr_cond['failure']:.3e}\n")

P(sys >= 2) = 8.015e-01
P(sys <= 1 ) = 1.985e-01



## Conditional probability given "one" component's survival

In [ ]:
for x in row_names:
    print(f"Eval P(sys | {x}=1)")
    pr_cond = tsum.get_comp_cond_sys_prob(
        rules_mat_surv,
        rules_mat_fail,
        probs,
        comps_st_cond = {x: 1},
        row_names = row_names,
        s_fun = s_fun,
        sys_surv_st = sys_surv_st
    )
    print(f"P(sys >= {sys_surv_st} | {x}=1) = {pr_cond['survival']:.3e}")
    print(f"P(sys <= {sys_surv_st-1} | {x}=1) = {pr_cond['failure']:.3e}\n")

# Get multi-state probabilities

## First load rules for all states

In [8]:
TSUMPATH = Path("tsum_res") 
rules_dict_mat_surv = {}
rules_dict_mat_fail = {}

for sys_surv_st in [1, 2]:  # either 1 or 2
    rules_mat_surv = torch.load(TSUMPATH / f"rules_geq_{sys_surv_st}.pt", map_location="cpu")
    rules_mat_surv = rules_mat_surv.to(device)
    rules_dict_mat_surv[sys_surv_st] = rules_mat_surv
    rules_mat_fail = torch.load(TSUMPATH / f"rules_leq_{sys_surv_st-1}.pt", map_location="cpu")
    rules_mat_fail = rules_mat_fail.to(device)
    rules_dict_mat_fail[sys_surv_st] = rules_mat_fail

## Marginal probability

In [9]:
cond_probs = tsum.get_comp_cond_sys_prob_multi(
                rules_dict_mat_surv,
                rules_dict_mat_fail,
                probs,
                comps_st_cond = {},
                row_names = row_names,
                s_fun = s_fun
            )

print("P(sys):", cond_probs)

P(sys): {0: 0.000425, 1: 0.19817, 2: 0.801405}


## Conditional probability given "one" component's survival

In [ ]:
for x in row_names:
    cond_probs = tsum.get_comp_cond_sys_prob_multi(
                    rules_dict_mat_surv,
                    rules_dict_mat_fail,
                    probs,
                    comps_st_cond = {x: 1},
                    row_names = row_names,
                    s_fun = s_fun
                )

    print(f"P(sys | {x}=1):", cond_probs)